In [4]:
import sqlite3
import geojson

In [5]:
def dict_factory(cursor, row):
    d = {}
    for idx,col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

def get_wildfires_by_bb(south, west, north, east):
    fires = None
    with sqlite3.connect('data/nbac.sqlite') as conn:
        conn.enable_load_extension(True)
        conn.execute('SELECT load_extension("mod_spatialite");')
        conn.row_factory = dict_factory
        q = f"""
            select
                year
                , nfireid
                , basrc
                , firemaps
                , firemapm
                , firecaus
                , burnclas
                , sdate
                , edate
                , afsdate
                , afedate
                , capdate
                , poly_ha
                , adj_ha
                , adj_flag
                , agency
                , bt_gid
                , version
                , comments
                , basrc_1
                , new_flg
                , asgeojson(GEOMETRY) as geom
                        from nbac
                        where ST_Intersects(
                                polygonfromtext(
                                    'POLYGON((
                            {west} {south},
                            {west} {north},
                            {east} {north},
                            {east} {south},
                            {west} {south}
                            ))')
                , GEOMETRY)
                limit 500
                """
        fires = conn.execute(q).fetchall()

        features = list()

        for row in fires:
            geom = geojson.loads(row['geom'])
            row.pop('geom')
            feature = geojson.Feature(geometry=geom, properties=row)
            features.append(feature)

        fireCollection = geojson.FeatureCollection(features)

        #fireCollection['crs'] = { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } }

        return fireCollection

In [6]:
east = -114.46294784545898
north = 55.17151537567731
south = 55.09300655867381
west = -114.60027694702148

In [7]:
west = -114.95171070098877
north = 55.13349468079407
east = -114.91737842559814
south = 55.11386325016275

In [8]:

west = -120.39764404296875
north = 55.88095201504089
east = -100.29901123046875
south = 49.243964309850995


In [9]:
west = -115.68878173828125
north = 55.66984035430873
east = -113.49151611328125
south = 54.41093863702367

In [10]:
q = f"""
select 
    year
    , nfireid
    , basrc
    , firemaps
    , firemapm
    , firecaus
    , burnclas
    , sdate
    , edate
    , afsdate
    , afedate
    , capdate
    , poly_ha
    , adj_ha
    , adj_flag
    , agency
    , bt_gid
    , version
    , comments
    , basrc_1
    , new_flg
    , centroid_x
    , centroid_y
    , asgeojson(GEOMETRY) as geom
    , Cluster as cluster
            from nbac
            where ST_Intersects(
                    polygonfromtext(
                        'POLYGON((
            {west} {south},
            {west} {north},
            {east} {north},
            {east} {south},
            {west} {south}
            ))'
        , 4326)
, GEOMETRY)
limit 2
                """
print(q)


select 
    year
    , nfireid
    , basrc
    , firemaps
    , firemapm
    , firecaus
    , burnclas
    , sdate
    , edate
    , afsdate
    , afedate
    , capdate
    , poly_ha
    , adj_ha
    , adj_flag
    , agency
    , bt_gid
    , version
    , comments
    , basrc_1
    , new_flg
    , centroid_x
    , centroid_y
    , asgeojson(GEOMETRY) as geom
    , Cluster as cluster
            from nbac
            where ST_Intersects(
                    polygonfromtext(
                        'POLYGON((
            -115.68878173828125 54.41093863702367,
            -115.68878173828125 55.66984035430873,
            -113.49151611328125 55.66984035430873,
            -113.49151611328125 54.41093863702367,
            -115.68878173828125 54.41093863702367
            ))'
        , 4326)
, GEOMETRY)
limit 2
                


In [11]:
f = get_wildfires_by_bb(south, west, north, east)

OperationalError: The specified module could not be found.


In [6]:
len(f['features'])

228

In [17]:
f['crs']

{'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}

In [35]:
fires.keys()

dict_keys(['type', 'features'])

In [36]:
len(fires['features'])

100